# Environment Setup

In [3]:
import os
from neo4j import GraphDatabase, basic_auth
import warnings
warnings.filterwarnings('ignore')

In [201]:
uri = "neo4j://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "your_password"))
session =  driver.session()

# Load Datasets

In [45]:
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import time

pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)

In [191]:
# Datasets loading csv
job_posting_glassdoor = pd.read_csv("Cleaned_DS_Jobs.csv")

In [26]:
if os.path.exists(directory_path):
    print("Detect sponsorship company lists datasets")
else:
    # 2023 sponsorship datasets
    company_names = []
    total_iterations = 1597

    for page_number in tqdm(range(1, total_iterations + 1), desc="Scraping Pages"):
        url = f'https://h1binfo.org/top/sponsors/2023?page={page_number}'
        response = requests.get(url)

        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            company_links = soup.find_all('a', class_='font-semibold text-blue-500 capitalize')
            company_names.extend([link.get_text().strip() for link in company_links])
        else:
            print(f"Failed to retrieve data from page {page_number}")

        time.sleep(2)

Scraping Pages: 100%|██████████████████| 1597/1597 [59:54<00:00,  2.25s/it]


In [82]:
len(company_names)

31912

In [34]:
# export to csv
# convert to pandas dataframe
# export to csv

company_name_sponsorship = pd.DataFrame(company_names)
company_name_sponsorship.colunams  = ['company_name']
company_name_sponsorship.to_csv('company_sponsorship.csv', sep='\t')

# Data Preprocessing

In [ ]:
job_posting_glassdoor

In [192]:
job_posting_glassdoor.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Type of ownership,Industry,Sector,Revenue,min_salary,max_salary,avg_salary,job_state,same_state,company_age,python,excel,hadoop,spark,aws,tableau,big_data,job_simp,seniority
0,Sr Data Scientist,137-171,Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst,"New York, NY","New York, NY",1001 to 5000 employees,Nonprofit Organization,Insurance Carriers,Insurance,Unknown / Non-Applicable,137,171,154,NY,1,27,0,0,0,0,1,0,0,data scientist,senior
1,Data Scientist,137-171,"Secure our Nation, Ignite your Future\n\nJoin ...",4.2,ManTech,"Chantilly, VA","Herndon, VA",5001 to 10000 employees,Company - Public,Research & Development,Business Services,$1 to $2 billion (USD),137,171,154,VA,0,52,0,0,1,0,0,0,1,data scientist,na
2,Data Scientist,137-171,Overview\n\n\nAnalysis Group is one of the lar...,3.8,Analysis Group,"Boston, MA","Boston, MA",1001 to 5000 employees,Private Practice / Firm,Consulting,Business Services,$100 to $500 million (USD),137,171,154,MA,1,39,1,1,0,0,1,0,0,data scientist,na
3,Data Scientist,137-171,JOB DESCRIPTION:\n\nDo you have a passion for ...,3.5,INFICON,"Newton, MA","Bad Ragaz, Switzerland",501 to 1000 employees,Company - Public,Electrical & Electronic Manufacturing,Manufacturing,$100 to $500 million (USD),137,171,154,MA,0,20,1,1,0,0,1,0,0,data scientist,na
4,Data Scientist,137-171,Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions,"New York, NY","New York, NY",51 to 200 employees,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,137,171,154,NY,1,22,1,1,0,0,0,0,0,data scientist,na


In [193]:
# Check sponsorship
def SponsorCheck(row):
    for company in company_names: 
        if row.lower() in company:
            return 'YES'
    return 'NO'

job_posting_glassdoor['Sponsorship'] = job_posting_glassdoor['Company Name'].apply(SponsorCheck)

In [194]:
job_posting_glassdoor.columns = job_posting_glassdoor.columns.str.replace(' ', '_').str.lower()

In [195]:
job_posting_glassdoor.columns

Index(['job_title', 'salary_estimate', 'job_description', 'rating',
       'company_name', 'location', 'headquarters', 'size', 'type_of_ownership',
       'industry', 'sector', 'revenue', 'min_salary', 'max_salary',
       'avg_salary', 'job_state', 'same_state', 'company_age', 'python',
       'excel', 'hadoop', 'spark', 'aws', 'tableau', 'big_data', 'job_simp',
       'seniority', 'sponsorship'],
      dtype='object')

In [196]:
output = job_posting_glassdoor[['job_title', 'rating', 'company_name', 'location', 'headquarters',
                               'type_of_ownership', 'industry', 'size', 'sector', 'company_age', 
                              'sponsorship']]

In [197]:
output.head()

,job_title,rating,company_name,location,headquarters,type_of_ownership,industry,size,sector,company_age,sponsorship
0,Sr Data Scientist,3.1,Healthfirst,"New York, NY","New York, NY",Nonprofit Organization,Insurance Carriers,1001 to 5000 employees,Insurance,27,YES
1,Data Scientist,4.2,ManTech,"Chantilly, VA","Herndon, VA",Company - Public,Research & Development,5001 to 10000 employees,Business Services,52,NO
2,Data Scientist,3.8,Analysis Group,"Boston, MA","Boston, MA",Private Practice / Firm,Consulting,1001 to 5000 employees,Business Services,39,YES
3,Data Scientist,3.5,INFICON,"Newton, MA","Bad Ragaz, Switzerland",Company - Public,Electrical & Electronic Manufacturing,501 to 1000 employees,Manufacturing,20,NO
4,Data Scientist,2.9,Affinity Solutions,"New York, NY","New York, NY",Company - Private,Advertising & Marketing,51 to 200 employees,Business Services,22,YES


In [198]:
output.to_csv('test.csv', index=False)

In [ ]:
# Use posgresql


# Neo4j - Import and Set up 

In [220]:
def show_result(query):
    result = session.run(query)
    print(result.data())
    

def show_table_result(query, session):
    result = session.run(query)
    df = pd.DataFrame([record.values() for record in result], columns=result.keys())
    return df

In [217]:
def load_data(driver, row):
    with driver.session() as session:
        session.write_transaction(_create_and_link, row)

def _create_and_link(tx, row):
    query = """
        MERGE (company:Company {name: $company_name})
        MERGE (sponsorship:Sponsorship {name: $sponsorship})
        MERGE (location:Location {name: $location})
        MERGE (job:JobTitle {name: $job_title, companyName: $company_name})
        MERGE (headquarters:Headquarters {name: $headquarters})
        MERGE (industryCompany:IndustryCompany {name: $industry})
        MERGE (rate:Rate {name: $rating})
        MERGE (ownership:Ownership {name: $type_of_ownership})
        MERGE (size:Size {name: $size})

        // Create relationships
        WITH company, job, sponsorship, location, headquarters, industryCompany, rate, ownership, size
        MERGE (company)-[:OFFERS]->(job)
        MERGE (company)-[:HAS_SPONSORSHIP]->(sponsorship)
        MERGE (job)-[:IN_LOCATION]->(location)
        MERGE (company)-[:LOCATED_AT]->(headquarters)
        MERGE (company)-[:WORKS_IN_INDUSTRY]->(industryCompany)
        MERGE (company)-[:HAS_RATE]->(rate)
        MERGE (company)-[:HAS_OWNERSHIP]->(ownership)

        // Additional relationships based on location and headquarters being the same, and job size
        WITH job, location, company, headquarters, size
        WHERE location.name = headquarters.name
        MERGE (job)-[:LOCATED_AT_HEADQUARTERS]->(headquarters)
        WITH job, size
        MERGE (job)-[:HAS_SIZE]->(size)
    """
    tx.run(query, row)

In [218]:
for index, row in output.iterrows():
    load_data(driver, row.to_dict())

In [225]:
query = """
        MATCH (job:JobTitle)-[:IN_LOCATION]->(location:Location),
        (job)-[:LOCATED_AT_HEADQUARTERS]->(headquarters:Headquarters),
        (company:Company)-[:OFFERS]->(job),
        (company)-[:HAS_RATE]->(rate:Rate),
        (company)-[:HAS_SPONSORSHIP]->(sponsorship:Sponsorship)
        WHERE location.name = headquarters.name AND toFloat(rate.name) > 4 AND sponsorship.name = 'YES'
        RETURN job.name, company.name, rate.name, location.name
        """

show_table_result(query, session)

,job.name,company.name,rate.name,location.name
0,"Manager / Lead, Data Science & Analytics",ABIOMED,4.1,"Danvers, MA"
1,"Manager / Lead, Data Science & Analytics",ABIOMED,4.1,"Danvers, MA"
2,Data Scientist,Klaviyo,4.8,"Boston, MA"
3,Data Science Software Engineer,Klaviyo,4.8,"Boston, MA"
4,Machine Learning Engineer,Klaviyo,4.8,"Boston, MA"
...,...,...,...,...
61,"Vice President, Biometrics and Clinical Data M...",Alector,4.8,"South San Francisco, CA"
62,Data Scientist,Maven Wave Partners,4.5,"Chicago, IL"
63,Data Scientist,Maven Wave Partners,4.5,"Chicago, IL"
64,Data Scientist,Demandbase,4.5,"San Francisco, CA"


In [230]:
driver.close()